In [1]:
!pip install pillow
!pip install numpy
!pip install opencv-python
!pip install imageio
!pip install ffmpeg-python


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.1 MB/s eta 0:00:00


In [1]:
import cv2
import os
import json
from ultralytics import YOLO

model = YOLO(r"D:\TCS\Road_Damage_Detection\Yolov8m\runforyolov8m\content\runforyolov8m\training_yolov8m\weights\best.pt")
input_path = "D:\TCS\Road_Damage_Detection\Video 1.mp4"

def is_video(path):
    return os.path.splitext(path)[1].lower() in [".mp4", ".avi", ".mov", ".mkv"]

def is_image(path):
    return os.path.splitext(path)[1].lower() in [".jpg", ".jpeg", ".png"]

results_data = {}

if is_image(input_path):
    img = cv2.imread(input_path)
    if img is None:
        raise ValueError("Could not read the image file")
    results = model(img, conf=0.3)
    detections = []
    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        confidences = r.boxes.conf.cpu().numpy()
        classes = r.boxes.cls.cpu().numpy()
        for box, conf, cls in zip(boxes, confidences, classes):
            x1, y1, x2, y2 = map(float, box)
            detections.append({
                "class": int(cls),
                "bbox": [x1, y1, x2, y2],
                "confidence": float(conf)
            })
    results_data["image"] = detections
    annotated = results[0].plot()
    cv2.imwrite("annotated_image.jpg", annotated)

elif is_video(input_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise ValueError("Could not open video")
    frame_idx = 0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter("annotated_video.mp4", fourcc, fps, (width, height))
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1
        results = model(frame, conf=0.3)
        detections = []
        for r in results:
            boxes = r.boxes.xyxy.cpu().numpy()
            confidences = r.boxes.conf.cpu().numpy()
            classes = r.boxes.cls.cpu().numpy()
            for box, conf, cls in zip(boxes, confidences, classes):
                x1, y1, x2, y2 = map(float, box)
                detections.append({
                    "class": int(cls),
                    "bbox": [x1, y1, x2, y2],
                    "confidence": float(conf)
                })
        results_data[f"frame_{frame_idx}"] = detections
        annotated_frame = results[0].plot()
        writer.write(annotated_frame)
    cap.release()
    writer.release()

with open("detection_results.json", "w") as f:
    json.dump(results_data, f, indent=2)


KeyboardInterrupt: 